In [ ]:
from semantic_search_to_json import Output_Analysis

Output_Analysis("How to play monopoly" , 'input_small.json')


In [6]:
from llm4_to_json import extract_outline_and_title
import pymupdf4llm
import json
import os

def parse_pdf_for_text_and_outline(docs_path=None):
    all_outlines = []
    text_and_meta = []
    docs_path = "/home/lord_grivous/Desktop/1Breal/AntonB/data/"
    for file in [file for file in os.listdir(docs_path) if file.endswith('.pdf')]:
        file_path = os.path.join(docs_path, file)

        md_text = pymupdf4llm.to_markdown(file_path, page_chunks=True)
        all_outlines.append(extract_outline_and_title(md_text))

        page_text = ""
        for pages in md_text:        
            page_text += pages["text"]
        text_and_meta.append({
            "text": page_text,
            "document": file_path,
        })

    return text_and_meta, all_outlines
    


In [7]:
text_and_meta, all_outlines =  parse_pdf_for_text_and_outline()


In [8]:
from fastembed.embedding import DefaultEmbedding
import numpy as np

# Load small embedding model (auto-downloads)
embedder = DefaultEmbedding(model_name="BAAI/bge-small-en-v1.5", cache_dir="emb_models")

def avg_embed(text: str) -> np.ndarray:
    embeddings = list(embedder.embed(text))
    return np.array(embeddings[0]) if embeddings else np.zeros(embedder.embedding_size)


In [9]:
avg_embeddings = [avg_embed(item["text"]) for item in text_and_meta]

In [16]:
from fastembed.embedding import DefaultEmbedding
import chromadb
from chromadb.config import Settings

# Setup Chroma DB
client = chromadb.Client(Settings(
    chroma_db_impl="duckdb+parquet",
    persist_directory="./chroma_db"
))

# Create collection
collection = client.get_or_create_collection(name="my_docs")

# Load embedder
embedder = DefaultEmbedding(model_name="BAAI/bge-small-en-v1.5")

# Your data
texts = [
    "How to play Monopoly",
    "Monopoly is a board game where players buy property.",
    "To cook pasta, boil water and add salt.",
]

# Add embeddings to Chroma
for i, text in enumerate(texts):
    vector = list(embedder.embed(text))[0]
    collection.add(
        documents=[text],
        embeddings=[vector],
        ids=[f"doc_{i}"]
    )

# Persist the DB (optional)
client.persist()


ValueError: [91mYou are using a deprecated configuration of Chroma.

[94mIf you do not have data you wish to migrate, you only need to change how you construct
your Chroma client. Please see the "New Clients" section of https://docs.trychroma.com/deployment/migration.
________________________________________________________________________________________________

If you do have data you wish to migrate, we have a migration tool you can use in order to
migrate your data to the new Chroma architecture.
Please `pip install chroma-migrate` and run `chroma-migrate` to migrate your data and then
change how you construct your Chroma client.

See https://docs.trychroma.com/deployment/migration for more information or join our discord at https://discord.gg/MMeYNTmh3x for help![0m